In [1]:
import tensorflow as tf
import pandas as pd
from keras.preprocessing import text, sequence
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Activation, Dropout
from keras import utils
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, LabelEncoder

/Users/niccolominetti/anaconda/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/niccolominetti/anaconda/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/niccolominetti/anaconda/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/niccolominetti/anaconda/lib/python3.5/site-packages

In [2]:
dataset = pd.read_csv("the-movies-dataset/movies_metadata.csv")

/Users/niccolominetti/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
dataset = dataset[dataset.genres != '[]']
dataset = dataset[dataset.overview != 'No overview found.']
dataset = dataset[dataset.overview != 'Released']

In [4]:
dataset = dataset[['overview', 'genres']]
dataset = dataset.dropna()

In [5]:
dataset['genres'] = dataset['genres'].map(lambda x: x.lstrip('[{').rstrip('}]'))
dataset['genres'] = dataset['genres'].map(lambda x: x.split("}, {"))
dataset['genres'] = dataset['genres'].map(lambda x: x[0])

In [6]:
reduced = dataset.groupby('genres').head(2000)
reduced = reduced.reset_index().drop('index', axis = 1)


In [7]:
reduced['genres'] = reduced['genres'].map(lambda x: [x])


In [8]:
print("Total Number of entries: ", len(reduced))
print("Train entries (80%): ", int(len(reduced) * .8))
print("Test entries (20%): ", int(len(reduced) * .2))

Total Number of entries:  21532
Train entries (80%):  17225
Test entries (20%):  4306


In [9]:
train_size = int(len(reduced) * .8)
train_entries = reduced['overview'][:train_size]
train_cat = reduced['genres'][:train_size]

In [10]:
test_entries = reduced['overview'][train_size:]
test_cat = reduced['genres'][train_size:]

In [39]:
vocab_size = 500
tokenize = text.Tokenizer(num_words=vocab_size)
tokenize.fit_on_texts(train_entries) 

In [40]:
x_train = tokenize.texts_to_matrix(train_entries)
x_test = tokenize.texts_to_matrix(test_entries)

In [41]:
tokenize.word_index

{'trekkies': 24065,
 'steward': 21771,
 'seamus': 25428,
 'remember': 1964,
 'miz': 26011,
 'paduan': 29645,
 'arouse': 14678,
 'daft': 18477,
 'sucker': 17512,
 "ex's": 33737,
 'limitation': 24219,
 'unliked': 35892,
 'motocross': 10348,
 'superstardom': 22941,
 'mcdhui': 18324,
 'manoeuvres': 22127,
 'restitution': 19815,
 'ezra': 21269,
 'souk': 45221,
 'kinds': 9783,
 'himself…': 43716,
 'wesley': 6920,
 'fins': 23607,
 'narcy': 48424,
 'pogue': 26970,
 'unrecognizable': 41337,
 'ziggy': 22877,
 'marcus’': 33610,
 'iturbi': 30321,
 'crap': 16163,
 'israeli': 3627,
 'flavorful': 47003,
 'qalli': 45842,
 'sexploitation': 24275,
 'bounin': 33560,
 'keenan': 10025,
 'psychos': 11770,
 'moomins': 44991,
 'aj': 25040,
 'smails': 21464,
 'daylong': 41808,
 'gartenberg': 42385,
 'goalie': 38928,
 'embroiled': 3549,
 'deckert': 36495,
 'ataru': 43731,
 'dashed': 20488,
 'potash': 45803,
 'rapunzel': 13580,
 'ulrik': 24041,
 "scorpion's": 45844,
 'jones': 1305,
 'abnormal': 23011,
 'tags': 1

In [42]:
encoder = MultiLabelBinarizer()
encoder.fit(train_cat)                     
y_train = encoder.transform(train_cat)
y_test = encoder.transform(test_cat)
encoder.classes_


array(["'id': 10402, 'name': 'Music'", "'id': 10749, 'name': 'Romance'",
       "'id': 10751, 'name': 'Family'", "'id': 10752, 'name': 'War'",
       "'id': 10769, 'name': 'Foreign'",
       "'id': 10770, 'name': 'TV Movie'", "'id': 12, 'name': 'Adventure'",
       "'id': 14, 'name': 'Fantasy'", "'id': 16, 'name': 'Animation'",
       "'id': 18, 'name': 'Drama'", "'id': 27, 'name': 'Horror'",
       "'id': 28, 'name': 'Action'", "'id': 35, 'name': 'Comedy'",
       "'id': 36, 'name': 'History'", "'id': 37, 'name': 'Western'",
       "'id': 53, 'name': 'Thriller'", "'id': 80, 'name': 'Crime'",
       "'id': 878, 'name': 'Science Fiction'",
       "'id': 9648, 'name': 'Mystery'", "'id': 99, 'name': 'Documentary'"],
      dtype=object)

In [43]:
model = []
model = Sequential()

#First Layer

model.add(Dense(512, input_shape=(vocab_size,)))    
model.add(Activation('relu'))

model.add(Dense((20)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',      
              optimizer='adam',                     
              metrics=['accuracy'])

history = model.fit(x_train, y_train,               
                    batch_size=32, 
                    epochs=2, 
                    verbose=1, 
                    validation_split=0.1)


score = model.evaluate(x_test, y_test,              
                       batch_size=32, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 15502 samples, validate on 1723 samples
Epoch 1/2
15502/15502 [==============================] - 3s - loss: 2.2875 - acc: 0.3061 - val_loss: 2.4869 - val_acc: 0.2577
Epoch 2/2
3680/4307 [========================>.....] - ETA: 0sTest score: 3.055779080726388
Test accuracy: 0.09821221267876726


$
$
$
Testing
$
$
$

In [34]:
labels = encoder.classes_
labels

array(["'id': 10402, 'name': 'Music'", "'id': 10749, 'name': 'Romance'",
       "'id': 10751, 'name': 'Family'", "'id': 10752, 'name': 'War'",
       "'id': 10769, 'name': 'Foreign'",
       "'id': 10770, 'name': 'TV Movie'", "'id': 12, 'name': 'Adventure'",
       "'id': 14, 'name': 'Fantasy'", "'id': 16, 'name': 'Animation'",
       "'id': 18, 'name': 'Drama'", "'id': 27, 'name': 'Horror'",
       "'id': 28, 'name': 'Action'", "'id': 35, 'name': 'Comedy'",
       "'id': 36, 'name': 'History'", "'id': 37, 'name': 'Western'",
       "'id': 53, 'name': 'Thriller'", "'id': 80, 'name': 'Crime'",
       "'id': 878, 'name': 'Science Fiction'",
       "'id': 9648, 'name': 'Mystery'", "'id': 99, 'name': 'Documentary'"],
      dtype=object)

In [44]:
test_string = "It is an animation movie"
test = tokenize.texts_to_matrix(test_string)

In [45]:
results = model.predict(test)[0]

In [46]:
best_score = 0
best_label = ''
for label in range(0, len(labels)):
    print(labels[label], ".. .. .. with Probability: "  , results[label])
    if(best_score < results[label]):
        best_label = labels[label]
        best_score = results[label]
        
print("\n\nBEST SCORE: ", best_label)

'id': 10402, 'name': 'Music' .. .. .. with Probability:  0.03768133
'id': 10749, 'name': 'Romance' .. .. .. with Probability:  0.045421712
'id': 10751, 'name': 'Family' .. .. .. with Probability:  0.025357604
'id': 10752, 'name': 'War' .. .. .. with Probability:  0.024953458
'id': 10769, 'name': 'Foreign' .. .. .. with Probability:  0.02444672
'id': 10770, 'name': 'TV Movie' .. .. .. with Probability:  0.028290758
'id': 12, 'name': 'Adventure' .. .. .. with Probability:  0.050251544
'id': 14, 'name': 'Fantasy' .. .. .. with Probability:  0.03506888
'id': 16, 'name': 'Animation' .. .. .. with Probability:  0.037823852
'id': 18, 'name': 'Drama' .. .. .. with Probability:  0.10038606
'id': 27, 'name': 'Horror' .. .. .. with Probability:  0.07095448
'id': 28, 'name': 'Action' .. .. .. with Probability:  0.07924929
'id': 35, 'name': 'Comedy' .. .. .. with Probability:  0.09824033
'id': 36, 'name': 'History' .. .. .. with Probability:  0.027691165
'id': 37, 'name': 'Western' .. .. .. with Pr